In [3]:
import transformers
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import datasets
from datasets import load_dataset
from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict


from typing import List, Optional, Tuple, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer

from utils import untuple
from model_utils import ModelWrapper

%load_ext autoreload
%autoreload 2

In [4]:
# test that hooks work
device = "cuda"
prompt = "Testing that hooks work"

mt = ModelWrapper("EleutherAI/pythia-160m", device = "cuda")
mt.register_layer_hooks()

toks = mt.tokenizer.encode(prompt, return_tensors = 'pt').to(device)

out= mt.model.generate(
    input_ids=toks,
    attention_mask = torch.ones_like(toks),
    max_new_tokens=10,
    top_p=1.0,
    output_hidden_states=True,
    return_dict_in_generate = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [12]:
assert torch.equal(out['hidden_states'][-1][11], mt.save_ctx['gpt_neox.layers.10'].activations)
assert torch.equal(out['hidden_states'][-1][12], mt.model.gpt_neox.final_layer_norm(mt.save_ctx['gpt_neox.layers.11'].activations))


True